# Transduction (sequence-to-sequence model)

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

from reservoirpy.datasets import japanese_vowels
from reservoirpy import set_seed, verbosity
from reservoirpy.observables import nrmse, rsquare

from sklearn.metrics import accuracy_score

set_seed(42)
verbosity(0)

In [ ]:
X_train, Y_train, X_test, Y_test = japanese_vowels()

In [ ]:
len(X_train), X_train[0], X_train[0].shape, X_train[0][0], X_train[0][0].shape, X_train[0][0][0], type(X_train[0][0][0])

In [ ]:
len(X_train), X_train[0].shape, X_train[1].shape

In [ ]:
len(X_train), X_train[0].T.shape, X_train[1].T.shape

In [ ]:
len(Y_train), Y_train[0].shape, Y_train[1].shape

In [ ]:
plt.imshow(X_train[0].T)
plt.title(f"A sample vowel of speaker {np.argmax(Y_train[0]) +1}")
plt.xlabel("Timesteps")
plt.ylabel("LPC (cepstra)")
plt.show()

plt.imshow(X_train[50].T)
plt.title(f"A sample vowel of speaker {np.argmax(Y_train[50]) +1}")
plt.xlabel("Timesteps")
plt.ylabel("LPC (cepstra)")
plt.show()

In [ ]:
sample_per_speaker = 30
n_speaker = 9
X_train_per_speaker = []

for i in range(n_speaker):
    X_speaker = X_train[i*sample_per_speaker: (i+1)*sample_per_speaker]
    X_train_per_speaker.append(np.concatenate(X_speaker).flatten())

plt.boxplot(X_train_per_speaker)
plt.xlabel("Speaker")
plt.ylabel("LPC (cepstra)")
plt.show()

In [ ]:
# repeat_target ensure that we obtain one label per timestep, and not one label per utterance.
X_train, Y_train, X_test, Y_test = japanese_vowels(repeat_targets=True)

In [ ]:
len(X_train), X_train[0], X_train[0].shape, X_train[0][0], X_train[0][0].shape, X_train[0][0][0], type(X_train[0][0][0])

In [ ]:
len(Y_train), Y_train[0], Y_train[0].shape, Y_train[0][0], Y_train[0][0].shape, Y_train[0][0][0], type(Y_train[0][0][0])

In [ ]:
from reservoirpy.nodes import Reservoir, Ridge, Input

In [ ]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = Ridge(ridge=1e-6)

model = [source >> reservoir, source] >> readout

In [ ]:
Y_pred = model.fit(X_train, Y_train, stateful=False, warmup=2).run(X_test, stateful=False)

In [ ]:
Y_pred_class = [np.argmax(y_p, axis=1) for y_p in Y_pred]
Y_test_class = [np.argmax(y_t, axis=1) for y_t in Y_test]

score = accuracy_score(np.concatenate(Y_test_class, axis=0), np.concatenate(Y_pred_class, axis=0))

print("Accuracy: ", f"{score * 100:.3f} %")

# Classification (sequence-to-vector model)

In [ ]:
X_train, Y_train, X_test, Y_test = japanese_vowels()

In [ ]:
from reservoirpy.nodes import Reservoir, Ridge, Input

In [ ]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = Ridge(ridge=1e-6)

model = source >> reservoir >> readout

In [ ]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])

In [ ]:
readout.fit(states_train, Y_train)

In [ ]:
Y_pred = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    y = readout.run(states[-1, np.newaxis])
    Y_pred.append(y)

In [ ]:
Y_pred_class = [np.argmax(y_p) for y_p in Y_pred]
Y_test_class = [np.argmax(y_t) for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy: ", f"{score * 100:.3f} %")